In [167]:
import pandas as pd
import matplotlib.pyplot as plt
import os

In [168]:
car_path = os.path.abspath(os.path.join(os.path.dirname("__file__"),'..', 'users'))
file_path = os.path.join(car_path, 'registro_partidas.csv')
datos = pd.read_csv(file_path)

In [169]:
def level_graphic():
    """ Porcentaje de partidas por nivel. """
    
    plt.clf()

    facil = datos[(datos['estado'] == 'finalizada') & (datos['nivel'] == 'Facil')]['nivel'].count()
    normal = datos[(datos['estado'] == 'finalizada') & (datos['nivel'] == 'Normal')]['nivel'].count()
    dificil = datos[(datos['estado'] == 'finalizada') & (datos['nivel'] == 'Dificil')]['nivel'].count()
    
    data_dibujo = [facil, normal, dificil]
    etiquetas = ['Facil', 'Normal', 'Dificil']
    plt.pie(data_dibujo, labels=etiquetas, autopct='%1.1f%%', shadow=True, startangle=90, labeldistance=1.1)
    plt.legend(etiquetas)
    plt.title('Porcentaje de partidas finalizadas por dificultad')
    plt.show()

In [170]:
def partidas_estado():
    """Gráfico que muestre el porcentaje de partidas por estado (fin, cancelada, error, timeout).
        También debe tener en cuenta las partidas que se terminaron inesperadamente o de forma repentina
        (partidas sin evento de fin)"""

    fin = datos[(datos['estado']) == 'finalizada']['estado'].count()
    #cierre por falla -> error
    #col_estado = datos.loc['estado']
    #cant_error =

    error = datos[(datos['evento']) == 'error']['estado'].count()
    timeout = datos[(datos['estado']) == 'timeout']['estado'].count()
    cancelada = datos[(datos['estado']) == 'cancelada']['estado'].count()

    data_graf = [fin, error, timeout, cancelada]
    etiquetas = ['fin', 'error', 'timeout', 'cancelada']
    plt.pie(data_graf, labels=etiquetas, autopct='%1.1f%%', shadow=True, startangle=90, labeldistance=1.1)
    plt.legend(etiquetas)
    plt.title('Porcentaje de partidas por estado')
    plt.show()

In [171]:
def porcentaje_finalizado_genero():
    """Gráfico que muestre el porcentaje de partidas finalizadas por género"""
    

    plt.clf()


    masculino = datos[(datos['estado'] == 'finalizada') & (datos['genero'] == 'Masculino')]['genero'].count()
    femenino = datos[(datos['estado'] == 'finalizada') & (datos['genero'] == 'Femenino')]['genero'].count()
    otro = datos[(datos['estado'] == 'finalizada') & (datos['genero'] == 'Otro')]['genero'].count()


    data_dibujo = [masculino, femenino, otro]
    etiquetas = ['Masculino', 'Femenino', 'Otro']


    plt.pie(data_dibujo, labels=etiquetas, autopct='%1.1f%%', shadow=True, startangle=90, labeldistance=1.1)
    plt.legend(etiquetas)
    plt.title('Porcentaje de partidas finalizadas por género')
    plt.show()

In [172]:
def cant_errores():
    """Cantidadad de errores por usuarie"""
    usuarios = datos['usuarie'].unique()
    cant_errores = [datos[(datos['usuarie'] == usuario) & (datos['estado'] == 'error')]['usuarie'].count() for usuario in usuarios]
    # for usuario in usuarios:
       # cant_errores.append(datos[(datos['usuarie'] == usuario) & (datos['estado'] == 'error')]['usuarie'].count())
    plt.bar(usuarios, cant_errores)
    plt.title('Cantidad de errores por usuarie')
    plt.show()

In [173]:
def cant_timeout():
    """Cantidad de tarjetas para las que el usuarie no dio respuesta (timeout)"""
    usuarios = datos['usuarie'].unique()
    cant_timeout = [datos[(datos['usuarie'] == usuario) & (datos['estado'] == 'timeout')]['usuarie'].count() for usuario in usuarios]
    plt.bar(usuarios, cant_timeout)
    plt.title('Cantidad de timeout por usuarie')
    plt.show()

In [174]:
def porcentaje_por_usuario():
    """Porcentaje de aciertos por usuarie."""
    usuarios = datos['usuarie'].unique()
    #cantidad de intentos por usuario
    cant_intento = [datos[(datos['usuarie'] == usuario) & (datos['evento'] == 'intento')]['usuarie'].count() for usuario in usuarios]
    #cantidad de ok por usuario
    cant_ok = [datos[(datos['usuarie'] == usuario) & (datos['estado'] == 'ok')]['usuarie'].count() for usuario in usuarios]

    #porcentaje de ok por usuario
    porcentaje_ok = [cant_ok[i]*100/cant_intento[i] for i in range(len(cant_intento))]

    plt.bar(usuarios, porcentaje_ok)
    plt.title('Porcentaje de aciertos por usuarie.')
    plt.show()

In [175]:
def tiempo_partida():
    """Duración total de la partida (desde que el usuarie empieza a jugar hasta que presiona
        salir o pasan las N rondas definidas para el nivel de dificultad actual)."""

    df = datos[['timestamp', 'id', 'evento']]
    values = ['inicio_partida', 'fin']
    
    #arma un DF eliminando las filas que no son inicio_partida ni fin
    df = df[df['evento'].isin(values)]
    
    #elimina los inicio_partida repetidos, es decir, descartamos las partidas finalizadas por errores y nos quedamos sólo con
    #las finalizadas correctamente  y reseteo los indices
    df_ev = df['evento']
    indices_drop = df_ev[df_ev == df_ev.shift(-1)]
    
    df = df.drop(indices_drop.index).reset_index(drop=True)
    
    #hago la diferencia entre una fila y la anterior para obtener los tiempos
    df['timestamp'] = df['timestamp'].diff()
    
    #me quedo con las columnas que me interesan
    result = df[['id', 'timestamp']]
    
    #elimino las filas que muestran el tiempo entre el fin de una partida y el comienzo de otra
    result = result.drop(result.index[result.index % 2 == 0])
    
    #renombro las columnas para claridad
    result.rename(columns={'id': 'id_partida', 'timestamp': 'duracion_partida'}, inplace=True)
    result = result.set_index('id_partida')

    #print(result)
    return result

In [180]:
def promedio_tiempo_partida():
    """Promedio de tiempo de partida general"""
    
    df = tiempo_partida()
    promedio = df.mean()
    print(promedio) #falta formatear este print

In [182]:
def grafico_total():
    """Grafico de lineas con todas las duraciones totales de las partidas finalizadas"""
    algo = [partida for partida in datos[(datos['estado'] == 'finalizada')]['tiempo-partida']]
    plt.plot(algo)
    plt.title('Duración total de las partidas finalizadas')
    plt.show()

In [ ]:
def top_palabras():
    """Top 10 de palabras que se encuentran primero de todas las partidas."""

    top = datos['respuesta'].value_counts()
    del(top['-'])
    top_10 = top[:10]

    
    
    top_10.plot(kind='bar')
    plt.title ('top 10 palabras')
    plt.ylabel('cantidad de repaticiones')
    plt.xlabel('palabras')
    fig = plt.gcf()
    fig.set_size_inches(11, 4)
    
    plt.show()

In [ ]:
def exitosa_mas_rapida():
    """ Tiempo de la respuesta exitosa más rápida."""
    
    df=datos[['timestamp', 'estado']]

    #encontrar el evento ok en la columna evento y restarle el timestamp anterior
    #agrego nueva columna para la resta de los tiempos
    df_2 = df.assign(tiempo_respuesta=0)

    #iterar sobre evento, restar timestamp y guardar resultado en teimpo_respuesta
    df_2['tiempo_respuesta'] = df_2['timestamp'] - df_2['timestamp'].shift(1).astype("Int64")
    #df_2.head()
    #df_2.keys()
    print(df_2[:12])

    #busco el minimo en tiempo_respuesta que cumpla con estado = ok
    #ordeno df_min por tiempo de respuesta ascendiente
    #df_min=df.sort_values('tiempo_respuesta', ascending=True)
    #df_min.head()

    group_tiempo=df_2.groupby('tiempo_respuesta')
    print(group_tiempo.min()[:1])

    #print(df.at[1,'tiempo_respuesta'])
